An attempt to start fresh on the Fractal dimension calculation problem (looking at the box counting and the generalised Renyi dimension on standard becnhamrking fucntionc (not the phsyical system)). Specifically, the Julia package FractalDimensions is implemented with no success so far. There is trouble with the type definnitons.

In [5]:
using Random, Statistics, Plots, StatsBase

# Function to generate synthetic fractal-like points at different resolutions
function generate_fractal_points(base_points::Int, levels::Int)
    points = []
    for level in 1:levels
        num_points = base_points * level  # Increasing resolution
        push!(points, sort(rand(num_points)))  # Random points with structure
    end
    return points
end

# Compute the probability distribution p_i for a given resolution
function compute_probabilities(points, ε)
    min_x, max_x = minimum(points), maximum(points)
    bins = min_x:ε:max_x
    # counts = histcounts(points, bins; normalize=true)
    # counts = StatsBase.histogram(points, bins; normalize=true)
    hist = fit(Histogram, points, bins)
    counts = hist.weights
    probs = counts / sum(counts)
    probs = probs[probs .> 0]  # Only consider nonzero probabilities
    return probs
end

# Compute the Renyi Dimension D_q for a set of resolutions
function compute_D_q(points_resolutions, q_values, eps_values)
    D_q_results = Dict(q => Float64[] for q in q_values)

    for ε in eps_values
        for q in q_values
            S_q_values = []

            for points in points_resolutions
                probs = compute_probabilities(points, ε)
                if q == 1
                    S_q = -sum(probs .* log.(probs))  # Information dimension (entropy-based)
                else
                    S_q = sum(probs .^ q) / (q - 1)  # Generalized q-dimension
                end
                push!(S_q_values, S_q)
            end

            # Fit log(S_q) vs log(ε) to get slope (scaling exponent)
            log_eps = log.(eps_values)
            log_S_q = log.(S_q_values)
            slope = -Statistics.cor(log_eps, log_S_q)  # Approximate scaling exponent

            push!(D_q_results[q], slope)
        end
    end

    return D_q_results
end

# Generate example data: 10 base points, increasing up to 100 points
Random.seed!(42)
points_resolutions = generate_fractal_points(10, 10)

# Define q-values (Rényi parameters) and resolution sizes ε
q_values = [-1, 0, 1, 2]
eps_values = exp10.(range(-2, stop=0, length=10))  # Log-spaced resolutions

# Compute D_q
D_q_results = compute_D_q(points_resolutions, q_values, eps_values)

# Plot results
plot()
for q in q_values
    plot!(eps_values, D_q_results[q], label="D_$(q)", lw=2, xscale=:log10, yscale=:log10)
end
xlabel!("Resolution ε")
ylabel!("D_q Scaling Exponent")
title!("Rényi Dimensions vs. Resolution")
display(plot())


DomainError: DomainError with -33.75:
log was called with a negative real argument but will only return a complex result if called with a complex argument. Try log(Complex(x)).

In [25]:
using StatsBase, Random, Statistics#, Plots

# Generate fractal-like points at different resolutions
function generate_fractal_points(base_points::Int, levels::Int)
    points = []
    for level in 1:levels
        num_points = base_points * level
        push!(points, sort(rand(num_points)))
    end
    return points
end

# Compute probability distribution
function compute_probabilities(points, ε)
    min_x, max_x = minimum(points), maximum(points)
    bins = min_x:ε:max_x
    hist = fit(Histogram, points, bins)
    counts = hist.weights
    probs = counts / sum(counts)  # Normalize to get probabilities
    probs = probs[probs .> 0]  # Remove zeros to avoid log issues
    return probs
end

# Compute Renyi Dimension D_q
function compute_D_q(points_resolutions, q_values, eps_values)
    D_q_results = Dict(q => Float64[] for q in q_values)

    for ε in eps_values
        for q in q_values
            S_q_values = []

            for points in points_resolutions
                probs = compute_probabilities(points, ε)
                if q == 1
                    S_q = -sum(probs .* log.(probs))  # Shannon entropy
                else
                    S_q = sum(probs .^ q) / (q - 1)  # Generalized dimension

                end
                push!(S_q_values, max(S_q, 1e-10))  # Avoid log(0) issues
            end

            # Fit log(S_q) vs log(ε) to get slope
            log_eps = log.(eps_values)
            log_S_q = log.(S_q_values)

            if any(isnan, log_S_q)
                @warn "NaNs detected in log(S_q), skipping this q-value"
                continue
            end

            slope = -Statistics.cor(log_eps, log_S_q)  # Scaling exponent

            push!(D_q_results[q], slope)
        end
    end

    return D_q_results
end

# Generate example data
Random.seed!(42)
points_resolutions = generate_fractal_points(10, 10)

# Define q-values and resolutions
q_values = [-1, 0, 1, 2]
eps_values = exp10.(range(-2, stop=0, length=10))  # Log-spaced resolutions

# Compute D_q
D_q_results = compute_D_q(points_resolutions, q_values, eps_values)

println("D_q_results: $D_q_results")
println("length D_q_results: $(length(D_q_results))")
println("D_q_results for q=$(q_values[2]): $(D_q_results[1])")
println("length D_q_results for q=$(q_values[2]): $(length(D_q_results[1]))")
println("epsilon values: $eps_values")


# # Plot results
# plt = Plots.plot(
#     legend=false,
#     xlabel="Resolution ε",
#     ylabel="D_q Scaling Exponent",
#     title="Rényi Dimensions vs. Resolution"
#     grid=true,
#     # size=(800, 600)
# )
# for q in q_values
#     plot!(eps_values, D_q_results[q], label="D_$(q)", lw=2, xscale=:log10, yscale=:log10)
# end

# display(plt)

using PyPlot

# Create figure
fig, ax = subplots(dpi=600)
ax.set_title("Rényi Dimensions vs. Resolution")
ax.set_xlabel(L"Resolution \varepsilon")
ax.set_ylabel(L"D_q Scaling Exponent")
ax.grid(true)
# figure(figsize=(6, 4))

# Loop over q-values and plot
for (q, D_q) in D_q_results
    println("plotting q=$q against D_q list=$D_q")
    ax.plot(eps_values, D_q, label=L"D_q (q=" * string(q) * ")")
end

# # Labels and title
# xlabel(L"Resolution \varepsilon")
# ylabel(L"D_q Scaling Exponent")
# title("Rényi Dimensions vs. Resolution")

# Legend and grid
ax.legend()
# grid(true, which="both", linestyle="--", alpha=0.6)

fig.savefig("D_q_fractal_test", dpi=600)

D_q_results: Dict(0 => [-9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17], -1 => [-9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17, -9.064864707369813e-17], 2 => [0.9025613871602634, 0.9172278082040843, 0.8809315968392789, 0.9004562257854437, 0.7693363967308098, 0.7846094790634497, 0.7294453294337111, 0.7257958648483434, NaN, -9.064864707369813e-17], 1 => [-0.8699717624743367, -0.8772492737410816, -0.8517328617377451, -0.8591074134899966, -0.7374049845165324, -0.7257504923995912, -0.6797981551414968, -0.6971044610238866, -9.064864707369813e-17, -9.064864707369813e-17])
length D_q_results: 4
D_q_results for q=0: [-0.8699717624743367, -0.

In [26]:
using StatsBase, Random, Statistics, PyPlot

# Generate fractal-like points at different resolutions
function generate_fractal_points(base_points::Int, levels::Int)
    points = []
    for level in 1:levels
        num_points = base_points * level
        push!(points, sort(rand(num_points)))  # Sort for consistency
    end
    return points
end

# Compute probability distribution
function compute_probabilities(points, ε)
    min_x, max_x = minimum(points), maximum(points)
    bins = min_x:ε:max_x
    hist = fit(Histogram, points, bins)
    counts = hist.weights
    probs = counts / sum(counts)  # Normalize to get probabilities
    return probs[probs .> 0]  # Remove zeros to avoid log issues
end

# Compute Renyi Dimension D_q
function compute_D_q(points_resolutions, q_values, eps_values)
    D_q_results = Dict(q => 0.0 for q in q_values)  # Store final D_q per q

    for q in q_values
        S_q_values = []

        for ε in eps_values
            S_q_eps = []

            for points in points_resolutions
                probs = compute_probabilities(points, ε)

                if q == 1
                    S_q = -sum(probs .* log.(probs))  # Shannon entropy
                else
                    S_q = sum(probs .^ q) / (q - 1)  # Generalized dimension
                end

                push!(S_q_eps, S_q)
            
            end

            push!(S_q_values, mean(S_q_eps))  # Average over different realizations

        end

        # Convert to log-log
        log_eps = log.(eps_values)
        log_S_q = log.(S_q_values)

        # Check for NaNs
        if any(isnan, log_S_q)
            @warn "NaNs detected in log(S_q), skipping q=$q"
            continue
        end

        # Fit log(S_q) vs log(ε) to get slope
        slope, _ = polyfit(log_eps, log_S_q, 1)  # Linear regression
        D_q_results[q] = -slope  # Fractal dimension

    end

    return D_q_results
end

# Generate example data
Random.seed!(42)
points_resolutions = generate_fractal_points(10, 10)

# Define q-values and resolutions
q_values = [-1, 0, 1, 2]
eps_values = exp10.(range(-2, stop=0, length=10))  # Log-spaced resolutions

# Compute D_q
D_q_results = compute_D_q(points_resolutions, q_values, eps_values)

println("D_q_results: $D_q_results")

# Plot results
fig, ax = subplots(dpi=600)
ax.set_title("Rényi Dimensions vs. Resolution")
ax.set_xlabel(L"Resolution \varepsilon")
ax.set_ylabel(L"D_q Scaling Exponent")
ax.grid(true)

# Loop over q-values and plot
for (q, D_q) in D_q_results
    ax.plot(eps_values, fill(D_q, length(eps_values)), label=L"D_q (q=" * string(q) * ")")
end

ax.legend()
fig.savefig("D_q_fractal_test_2.png", dpi=600)


DomainError: DomainError with -1042.98:
log was called with a negative real argument but will only return a complex result if called with a complex argument. Try log(Complex(x)).

## Using FractalDimensions Package

### Correlation Sum

using FractalDimensions.correlationsum

In [51]:
function log_range(base::Float64, start_power::Float64, end_power::Float64, length::Int)
    powers = range(start_power, stop=end_power, length=length)
    values = base .^ powers
    return values
end

log_range (generic function with 1 method)

In [54]:
function create_gaussian_dos(energy_range::Vector{Float64}, center::Float64, width::Float64, amplitude::Float64=1.0)
    total_dos = amplitude .* exp.(-((energy_range .- center).^2) ./ (2 * width^2))
    return total_dos
end

function create_weierstrass_curve(energy_range::Vector{Float64}, a::Float64, b::Int, n_terms::Int)
    """
    Generate a Weierstrass function for given parameters.

    Args:
        energy_range: The x-axis values (1D array).
        a: The coefficient (0 < a < 1).
        b: The base of exponents (positive odd integer).
        n_terms: The number of terms in the series.

    Returns:
        weierstrass_curve: The generated Weierstrass function values (1D array).
    """
    weierstrass_curve = zeros(length(energy_range))

    for i in 1:length(energy_range)
        x = energy_range[i]
        for n in 0:n_terms
            weierstrass_curve[i] += a^n * cos(b^n * π * x)
        end
    end

    return weierstrass_curve
end

function plt_raw_function(energy_range::Vector{Float64}, f::Vector{Float64}, function_name::String)
    fig, ax = subplots()
    ax.set_title("$function_name function")
    ax.set_xlabel("x")
    ax.set_ylabel("f")
    ax.grid(true)

    ax.plot(energy_range, f, lw=1)

    fig.savefig("raw_$(function_name)_plt.png", dpi=600)
    PyPlot.close()
end

gaussian_resolution = 5000
gaussian_energy_range = collect(range(-5.0, 5.0, gaussian_resolution))
center = 0.0
width = 1.0
amplitude = 3.0
gaussian_f = create_gaussian_dos(gaussian_energy_range, center, width, amplitude)
gauss_name = "Gaussian"

weier_resolution = 50000
weierstrass_energy_range = collect(range(0.0, 1.0, weier_resolution))
weier_amplitude = 0.6
weier_b = 5
n_terms = 2
weierstrass_f = create_weierstrass_curve(weierstrass_energy_range, weier_amplitude, weier_b, n_terms)
weier_name = "Weierstrauss"

plt_raw_function(weierstrass_energy_range, weierstrass_f, weier_name)
plt_raw_function(gaussian_energy_range, gaussian_f, gauss_name)


In [79]:
using DynamicalSystemsBase # for simulating dynamical systems
using CairoMakie           # for plotting

henon_rule(x, p, n) = SVector(1.0 - p[1]*x[1]^2 + x[2], p[2]*x[1])
u0 = zeros(2)
p0 = [1.4, 0.3]
henon = DeterministicIteratedMap(henon_rule, u0, p0)

X, t = trajectory(henon, 20_000; Ttr = 100)
# Plots.scatter(X[:, 1], X[:, 2]; color = ("black", 0.01), markersize = 4)

println(typeof(X))
println(typeof(t))


StateSpaceSet{2, Float64, SVector{2, Float64}}
StepRange{Int64, Int64}


In [73]:
using FractalDimensions

function plt_fractal_calc(log_eps::Vector{Float64}, log_counta::Vector{Float64}, function_name::String)
    fig, ax = subplots()
    ax.set_title("$function_name function")
    ax.set_xlabel("log_eps")
    ax.set_ylabel("log_counta")
    ax.grid(true)

    ax.scatter(log_eps, log_counta, lw=1)

    fig.savefig("fractal_plt_$(function_name).png", dpi=600)
    PyPlot.close()
end

# X = gaussian_f
X = weierstrass_f
println(typeof(X))

state_space_X = StateSpaceSet(X)

# eps = collect(range(100,4000,50))
eps = log_range(5.0, 1.5, 5.0, 36)
r0 = maximum(eps)
println("max epsilon (scale): $r0")

# # Simple correlation sum
# c_q_eps = FractalDimensions.correlationsum(X, eps; w = 0, norm = Euclidean(), q = 2)

# box-assisted correlation sum
good_prism = prismdim_theiler(X)
println("good prism: $good_prism")
P = good_prism
c_q_eps = FractalDimensions.boxed_correlationsum(state_space_X, eps, r0=r0, P=P)

# # generalised FractalDimensions
# c_q_eps = generalized_dim(state_space_X)

println("mean c_q_eps: $(mean(c_q_eps))")

# c_q_eps_log = log.(c_q_eps)
# log_eps = log.(eps)

# plt_fractal_calc(c_q_eps_log, log_eps, weier_name)

Vector{Float64}
max epsilon (scale): 3125.0
good prism: 1


MethodError: MethodError: no method matching boxed_correlationsum_2(::Vector{SVector{1, Int64}}, ::Vector{Vector{Int64}}, ::StateSpaceSet{1, Float64, SVector{1, Float64}}, ::Vector{Float64}; r0::Float64)
This error has been manually thrown, explicitly, so the method may exist but be intentionally marked as unimplemented.

Closest candidates are:
  boxed_correlationsum_2(::Any, ::Any, ::Any, ::Any; norm, w, show_progress) got unsupported keyword argument "r0"
   @ FractalDimensions ~/.julia/packages/FractalDimensions/SQX7P/src/corrsum_based/correlationsum_boxassisted.jl:166


In [2]:
t2_range = collect(range(0.0, 1.0, 11))
golden_asymptotes = Float64[]
silver_asymptotes = Float64[]

for t2 in t2_range
    g_range = floor.(Int, collect(range(1,20,20)))
    s_range = floor.(Int, collect(range(1,11,11)))
    golden_lengths = calc_golden_length(g_range)
    silver_lengths = calc_silver_length(s_range)

    t1=1.0
    t2=t2
    t3=2.0

    golden_w, golden_norm_w = w_metric_calc(golden_lengths, "golden", t1, t2, t3)
    silver_w, silver_norm_w = w_metric_calc(silver_lengths, "silver", t1, t2, t3)

    golden_fit, golden_asymptote = find_asymptote(golden_lengths, golden_norm_w)
    silver_fit, silver_asymptote = find_asymptote(silver_lengths, silver_norm_w)

    push!(golden_asymptotes, golden_asymptote)
    push!(silver_asymptotes, silver_asymptote)
end

Plots.scatter(
    t2_range,
    golden_asymptotes
)

Plots.scatter!(
    t2_range,
    silver_asymptotes
)

UndefVarError: UndefVarError: `calc_golden_length` not defined in `Main`
Suggestion: check for spelling errors or missing imports.